# 异步IO-Coroutine和Task

## 参考文献

* https://docs.python.org/3/library/asyncio-task.html

In [1]:
import asyncio
import time
import datetime

## Coroutine

通过`async`和`await`实现asyncio应用。

In [2]:
async def hello():
    print('Hello')
    await asyncio.sleep(1)
    print('World!')

`Croutine`对象

In [3]:
hello()

<coroutine object hello at 0x7ffd403fd830>

In [4]:
await hello()

Hello
World!


In [5]:
async def say_after(delay: int, message: str):
    await asyncio.sleep(delay)
    print(message)

In [6]:
async def main():
    print(f'Start at {time.strftime("%X")}')
    await say_after(1, 'Hello')
    await say_after(2, 'World!')
    
    # 运行时间3秒
    print(f'Finished at {time.strftime("%X")}')

In [7]:
await main()

Start at 14:21:20
Hello
World!
Finished at 14:21:23


`asyncio.create_task`将`coroutine`作为`Task`并行运行。

In [8]:
async def main():
    task1 = asyncio.create_task(say_after(1, 'Hello'))
    task2= asyncio.create_task(say_after(2, 'World!'))
    
    print(f'Start at {time.strftime("%X")}')
    await task1
    await task2
    
    # 运行时间2秒
    print(f'Finished at {time.strftime("%X")}')

In [9]:
await main()

Start at 14:21:23
Hello
World!
Finished at 14:21:25


## Awaitables

当对象是`Awaitable`对象时，可以通过`await`表达式。

三种主要Awaitablel类型：`Coroutine`、`Task`和`Future`。

### Coroutine

In [10]:
async def nested():
    return 42

In [11]:
nested()

<coroutine object nested at 0x7ffd28a549e0>

* `async def` 定义`coroutine`函数
* 调用`couroutine`函数，返回`coroutine`对象

### Task

Task用来并发的调度`Coroutine`.

通过`asyncio.create_taks()`可以将`Coroutine`封装成`Task`，`coroutine`调度后自动执行。

In [12]:
task = asyncio.create_task(nested())
await task

42

### Future

Future是底层`awaitable`对象，表示异步操作的结果。

In [13]:
async def delay(seconds: int, future: asyncio.Future):
    await asyncio.sleep(seconds)
    future.set_result('200')


async def main():
    loop = asyncio.get_event_loop()
    future = loop.create_future()
    asyncio.create_task(delay(1, future))

    # wait future done.
    await future
    print(f'future done, result {future.result()}')


In [14]:
await main()

future done, result 200


## Sleeping

`coroutine asyncio.sleep(delay, result=None)`

* 如果提供了`result`参数，结束时返回给调用者
* `sleep`会挂起当前任务，允许其他任务执行

## Running Task Cooncurrently

In [26]:
async def factorial(name: str, number: int):
    f = 1
    for i in range(2, number+1):
        print(f'Task {name}: compute factorial({number}), concurrently i={i}...')
        f *= i
        await asyncio.sleep(1)
        
    print(f'Task {name}: factorial({number}) = {f} ')
    return f

In [27]:
async def main():
    L = await asyncio.gather(
        factorial('A', 2),
        factorial('B', 3),
        factorial('C', 4)
    )
    print(L)

In [30]:
await main()

Task A: compute factorial(2), concurrently i=2...
Task B: compute factorial(3), concurrently i=2...
Task C: compute factorial(4), concurrently i=2...
Task A: factorial(2) = 2 
Task B: compute factorial(3), concurrently i=3...
Task C: compute factorial(4), concurrently i=3...
Task B: factorial(3) = 6 
Task C: compute factorial(4), concurrently i=4...
Task C: factorial(4) = 24 
[2, 6, 24]


## Timeout

In [34]:
async def eternity():
    await asyncio.sleep(3600)
    print('yay!')

In [35]:
async def main():
    try:
        await asyncio.wait_for(eternity(), timeout=1)
    except asyncio.TimeoutError:
        print('Timeout')

In [36]:
await main()

Timeout


## Introspection

In [39]:
asyncio.create_task(eternity())

<Task pending coro=<eternity() running at <ipython-input-34-d201bdf6629b>:1>>

当前任务

In [44]:
asyncio.current_task()

所有的任务

In [43]:
asyncio.all_tasks()

{<Task pending coro=<eternity() running at <ipython-input-34-d201bdf6629b>:2> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7ffd205a1310>()]>>}

## Task

In [49]:
async def cancel_me():
    print(f'cancel_me(): before sleep')
    
    try:
        await asyncio.sleep(3600)
    except asyncio.CancelledError:
        print(f'cancel_me(): cancle sleep')
        raise
    finally:
        print(f'cancel_me(): after sleep')

In [50]:
async def main():
    task = asyncio.create_task(cancel_me())
    await asyncio.sleep(1)
    
    task.cancel()
    
    try:
        await task
        
    except asyncio.CancelledError:
        print('main(): cancel_me is cancelled now.')

In [51]:
await main()

cancel_me(): before sleep
cancel_me(): cancle sleep
cancel_me(): after sleep
main(): cancel_me is cancelled now.
